## Updated at 24.10.2023

In [1]:
import psycopg2.extras
import psycopg2
from os import environ
import pandas as pd

postgres_config = {
    "host":environ["DB_HOST"],
    "port":environ["DB_PORT"],
    "user": environ["DB_USER"],
    "password": environ["DB_PASSWORD"],
    "database": environ["DB_NAME"],
}


## Players from teams playing against bad teams in the next games

In [2]:
query = """
select * from team_future_opponents_ratings order by overall_opponent_defending_rating desc
"""
pg_connection = psycopg2.connect(**postgres_config)

# first we have to fetch the necessary information from the postgres universe table.
with pg_connection.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as pg_cur:
    pg_cur.execute(query)
    df = pd.DataFrame(pg_cur.fetchall())

pg_connection.close()

playing_against = dict(zip(df['team'], df['opponents']))
teams_selected = tuple(df.loc[:5, 'team'].to_list())
print(teams_selected)

query = """
select 
    player_name,
    team,
    total_minutes_played,
    "xTotal", "xG", "xA", key_passes, shots, goals, assists,
    latest_match
from (
    select 
        row_number() over (partition by team order by "xTotal" desc) as row_num,
        *
    from player_form 
    where team in %(teams_selected)s
)
order by "xTotal" desc
"""
pg_connection = psycopg2.connect(**postgres_config)

# first we have to fetch the necessary information from the postgres universe table.
with pg_connection.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as pg_cur:
    pg_cur.execute(query, {"teams_selected":teams_selected})
    df = pd.DataFrame(pg_cur.fetchall())

df['playing_against'] = df['team'].apply(lambda x: playing_against[x])
df

('Bournemouth', 'Crystal Palace', 'Brentford', 'Arsenal', 'Brighton', 'Burnley')


,player_name,team,total_minutes_played,xTotal,xG,xA,key_passes,shots,goals,assists,latest_match,playing_against
0,Bryan Mbeumo,Brentford,540.0,3.982941,1.885310,2.097631,11.0,21.0,2.0,2.0,2023-11-04,"{Liverpool,Arsenal,Luton,Brighton,""Sheffield U..."
1,Estupiñán,Brighton,501.0,3.545141,0.958143,2.586997,13.0,8.0,1.0,3.0,2023-09-30,"{""Sheffield United"",""Nottingham Forest"",Chelse..."
2,Bukayo Saka,Arsenal,517.0,3.414102,2.175665,1.238438,8.0,7.0,2.0,2.0,2023-11-04,"{Burnley,Brentford,""Wolverhampton Wanderers"",L..."
3,Solly March,Brighton,496.0,3.056432,2.124008,0.932423,5.0,11.0,2.0,1.0,2023-10-21,"{""Sheffield United"",""Nottingham Forest"",Chelse..."
4,Eberechi Eze,Crystal Palace,480.0,2.961947,1.495185,1.466763,11.0,15.0,1.0,1.0,2023-11-04,"{Everton,Luton,""West Ham"",Bournemouth,Liverpoo..."
5,Yoane Wissa,Brentford,511.0,2.915155,2.072619,0.842536,7.0,10.0,1.0,1.0,2023-11-04,"{Liverpool,Arsenal,Luton,Brighton,""Sheffield U..."
6,Dominic Solanke,Bournemouth,538.0,2.864386,2.592166,0.272219,3.0,16.0,2.0,0.0,2023-11-04,"{""Newcastle United"",""Sheffield United"",""Aston ..."
7,Martin Odegaard,Arsenal,530.0,2.585332,1.369988,1.215345,13.0,12.0,2.0,1.0,2023-10-21,"{Burnley,Brentford,""Wolverhampton Wanderers"",L..."
8,Lyle Foster,Burnley,528.0,2.544701,1.829836,0.714865,5.0,8.0,3.0,2.0,2023-10-21,"{Arsenal,""West Ham"",""Sheffield United"",""Wolver..."
9,Kaoru Mitoma,Brighton,491.0,2.503852,2.319825,0.184028,3.0,11.0,2.0,0.0,2023-11-04,"{""Sheffield United"",""Nottingham Forest"",Chelse..."


## The best players based on their recent form (past 6 games)

In [3]:

query = """
select
    player_name,
    team,
    total_minutes_played,
    "xTotal", "xG", "xA", key_passes, shots, goals, assists,
    latest_match, team_played_against
from player_form
order by "xTotal" desc
"""
pg_connection = psycopg2.connect(**postgres_config)

# first we have to fetch the necessary information from the postgres universe table.
with pg_connection.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as pg_cur:
    pg_cur.execute(query)
    df = pd.DataFrame(pg_cur.fetchall())

pg_connection.close()
df.head(20)

,player_name,team,total_minutes_played,xTotal,xG,xA,key_passes,shots,goals,assists,latest_match,team_played_against
0,Mohamed Salah,Liverpool,531.0,7.446730,4.606679,2.840051,13.0,20.0,6.0,0.0,2023-11-05,"{Luton,""Nottingham Forest"",Everton,Brighton,To..."
1,Cole Palmer,"Chelsea,Manchester City",455.0,6.217812,3.292164,2.925648,11.0,15.0,3.0,2.0,2023-11-06,"{Tottenham,Chelsea,Chelsea,Burnley,Fulham,Chel..."
2,Darwin Núñez,Liverpool,407.0,5.984304,4.841980,1.142324,5.0,23.0,2.0,2.0,2023-11-05,"{Luton,""Nottingham Forest"",Everton,Brighton,""W..."
3,Ollie Watkins,Aston Villa,540.0,5.866912,3.992847,1.874065,10.0,19.0,5.0,3.0,2023-11-05,"{""Nottingham Forest"",Luton,""West Ham"",""Wolverh..."
4,Callum Wilson,Newcastle United,399.0,5.665449,5.665449,0.000000,0.0,14.0,5.0,0.0,2023-11-04,"{Arsenal,""Wolverhampton Wanderers"",""Crystal Pa..."
5,Alexander Isak,Newcastle United,359.0,5.348127,4.914274,0.433853,4.0,15.0,4.0,0.0,2023-10-21,"{""Crystal Palace"",""West Ham"",Burnley,""Sheffiel..."
6,Erling Haaland,Manchester City,500.0,5.316227,3.848984,1.467243,4.0,13.0,4.0,1.0,2023-11-04,"{Bournemouth,""Manchester United"",Brighton,Arse..."
7,Son Heung-Min,Tottenham,503.0,4.902761,3.221871,1.680891,10.0,17.0,5.0,1.0,2023-11-06,"{Chelsea,""Crystal Palace"",Fulham,Luton,Liverpo..."
8,Hee-Chan Hwang,Wolverhampton Wanderers,500.0,4.308441,2.857623,1.450818,10.0,10.0,3.0,2.0,2023-11-04,"{""Sheffield United"",""Newcastle United"",Bournem..."
9,Jarrod Bowen,West Ham,540.0,4.245216,3.146869,1.098348,4.0,19.0,4.0,0.0,2023-11-04,"{Brentford,Everton,""Aston Villa"",""Newcastle Un..."


## The recent team form (past 6 games) ordered by the best defence lately to the worst one

In [4]:

query = """
select
    *
from team_form
order by defending_rating asc
"""
pg_connection = psycopg2.connect(**postgres_config)

# first we have to fetch the necessary information from the postgres universe table.
with pg_connection.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as pg_cur:
    pg_cur.execute(query)
    df = pd.DataFrame(pg_cur.fetchall())

pg_connection.close()
df

,team,xG,xGA,xpts,attack_home,attack_away,defence_home,defence_away,matches_home,matches_away,avg_home_att,avg_away_att,avg_home_def,avg_away_def,defending_rating,attacking_rating
0,Manchester City,11.086610,4.111140,12.1586,4.701839,6.384771,2.335969,1.775171,3.0,3.0,1.567280,2.128257,0.778656,0.591724,1.370380,3.695537
1,Newcastle United,17.348790,4.799402,15.3024,7.942070,9.406720,2.425272,2.374130,3.0,3.0,2.647357,3.135573,0.808424,0.791377,1.599801,5.782930
2,Fulham,6.178952,6.261064,7.8718,4.337913,1.841039,2.933514,3.327550,3.0,3.0,1.445971,0.613680,0.977838,1.109183,2.087021,2.059651
3,Aston Villa,11.607560,6.392856,11.5747,7.325430,4.282130,2.840929,3.551927,3.0,3.0,2.441810,1.427377,0.946976,1.183976,2.130952,3.869187
4,Arsenal,11.432001,6.584259,9.6381,5.764418,5.667583,3.097886,3.486373,3.0,3.0,1.921473,1.889194,1.032629,1.162124,2.194753,3.810667
5,Everton,10.778752,7.412699,10.4098,7.436197,3.342556,2.934421,4.478278,3.0,3.0,2.478732,1.114185,0.978140,1.492759,2.470900,3.592917
6,Chelsea,12.652470,7.479203,11.5514,4.633550,8.018920,4.674070,2.805133,3.0,3.0,1.544517,2.672973,1.558023,0.935044,2.493068,4.217490
7,Liverpool,15.805110,8.035766,12.9932,9.132580,6.672530,2.274567,5.761200,3.0,3.0,3.044193,2.224177,0.758189,1.920400,2.678589,5.268370
8,Brighton,8.831572,8.140229,8.6248,5.697160,3.134412,4.652653,3.487576,3.0,3.0,1.899053,1.044804,1.550884,1.162525,2.713410,2.943857
9,Brentford,11.825390,8.775036,10.0700,6.560310,5.265080,4.963149,3.811887,3.0,3.0,2.186770,1.755027,1.654383,1.270629,2.925012,3.941797


## Show players with good overall stats

In [5]:

query = """
select
    id, player_name, games, time, goals, assists,
    "xG", "xA", "xG" + "xA" as "xTotal", 
    ("xG" + "xA") / games as "xTotal_per_game", 
    "xGChain" / games as "xGChain_per_game"
from player
where games > 3
order by "xTotal_per_game" desc
"""
pg_connection = psycopg2.connect(**postgres_config)

# first we have to fetch the necessary information from the postgres universe table.
with pg_connection.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor) as pg_cur:
    pg_cur.execute(query)
    df = pd.DataFrame(pg_cur.fetchall())

pg_connection.close()
df.head(30)

,id,player_name,games,time,goals,assists,xG,xA,xTotal,xTotal_per_game,xGChain_per_game
0,8260,Erling Haaland,11,945,11,2,11.495323,2.592708,14.088030,1.280730,1.008493
1,1250,Mohamed Salah,11,975,8,4,7.727359,5.920990,13.648349,1.240759,1.015332
2,8865,Ollie Watkins,11,969,5,5,6.405120,3.135984,9.541104,0.867373,0.927776
3,10720,Darwin Núñez,10,503,4,3,6.630821,1.727263,8.358083,0.835808,0.859620
4,10048,Nicolas Jackson,10,758,5,0,7.551804,0.735105,8.286908,0.828691,0.832501
5,5232,Alexander Isak,8,507,6,0,6.077839,0.433853,6.511693,0.813962,0.722998
6,6552,Bryan Mbeumo,11,990,6,2,5.779258,2.569817,8.349075,0.759007,0.723178
7,468,Callum Wilson,10,475,7,0,7.311268,0.015900,7.327168,0.732717,0.640357
8,8497,Cole Palmer,9,509,3,2,3.434348,2.985595,6.419943,0.713327,0.558448
9,453,Son Heung-Min,11,929,8,1,4.824765,3.000118,7.824883,0.711353,0.875374
